### Часть 2. Тематическое моделирование
Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить? Эта часть задания может быть сделана с использованием gensim.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse
warnings.filterwarnings('ignore')

%matplotlib inline

In [10]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:16, 2642.43it/s]


In [22]:
text = [] 
for row in responses:
    text.append(row['text'])
len(text)

153499

### Препроцессинг

In [14]:
import re
from nltk.corpus import stopwords

In [17]:
from pymystem3 import Mystem
m = Mystem()

In [26]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Unicorn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
mystopwords = stopwords.words('russian') + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-я]+")


def words_only(text):
    return " ".join(ru_words.findall(text))


def lemmatize(text):
    try:
        #return "".join(m.lemmatize(text)).strip()  
        return " ".join([pymorphy2_analyzer.parse(word)[0].normal_form for word in text.split()])
    except:
        return " "


def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

In [32]:
text1 = []
for item in text:
    text1.append(preprocess(item))

In [33]:
text1[15]

'произвести платёж банкомат г ростов дон адрес добровольский офис банк отп банк первый платёж второй платёж однако первый платёж поступить второй платёж практически моментально зачислить оплата производиться карта банкомат дать ситуация написать письмо адрес июнь г однако никакой ответ последовать письмо точный реквизит услуга скан чековый просить решить проблема оперативно комментарий представитель банк детальный информация изложить необходимость скан повторно выслать'

## Генсим

### TfidfModel

In [36]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim import similarities

In [44]:
text_list = [text.split() for text in text1[:10]]

In [43]:
dictionary = Dictionary(text_list)
dictionary

In [42]:
corpus = [dictionary.doc2bow(text) for text in text_list[:10]]
corpus

[[(0, 3),
  (1, 3),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 2),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 3),
  (15, 1),
  (16, 2),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 4),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 3),
  (65, 2),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 3),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1)

In [45]:
text_list = [text.split() for text in text1]
dictionary = Dictionary(text_list)
corpus = [dictionary.doc2bow(text) for text in text_list]

In [46]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [47]:
corpus_tfidf

### Латентно-семантический анализ (LSA, LSI)


Сингулярное разложение: $M'_k = U \Sigma_k V^t_k$

* $M$ – матрица слово-документ
* $U$ – матрица документ-тема
* $k$ – число тем
* $V$ - матрица слово-тема

In [49]:
from gensim.models import lsimodel

In [50]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=30)

Wall time: 1min 31s


In [55]:
lsi.show_topics(10)

[(0,
  '0.197*"карта" + 0.135*"кредит" + 0.117*"деньга" + 0.111*"заявление" + 0.108*"счёт" + 0.103*"банкомат" + 0.102*"вклад" + 0.102*"отделение" + 0.098*"сбербанк" + 0.098*"сумма"'),
 (1,
  '0.366*"кредит" + -0.361*"вклад" + -0.226*"банкомат" + -0.180*"карта" + 0.169*"страховка" + 0.145*"задолженность" + 0.142*"платёж" + 0.140*"погашение" + -0.125*"деньга" + 0.117*"звонок"'),
 (2,
  '-0.518*"вклад" + 0.270*"банкомат" + 0.209*"карта" + 0.182*"сбербанк" + 0.134*"заявление" + -0.121*"очередь" + 0.116*"операция" + -0.114*"кредит" + -0.114*"договор" + 0.113*"средство"'),
 (3,
  '-0.265*"вклад" + -0.219*"сумма" + 0.195*"номер" + -0.185*"руб" + 0.180*"заявка" + -0.159*"погашение" + -0.158*"платёж" + -0.155*"договор" + 0.150*"звонок" + -0.145*"рубль"'),
 (4,
  '0.319*"вклад" + 0.275*"номер" + -0.266*"карта" + -0.210*"страховка" + 0.195*"звонок" + -0.187*"заявка" + 0.143*"ваш" + 0.142*"телефон" + -0.138*"кредит" + -0.129*"лимит"'),
 (5,
  '0.480*"банкомат" + -0.361*"карта" + 0.158*"купюра" + 0

In [52]:
%%time
lsi10 = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10)

Wall time: 1min 30s


In [54]:
lsi.show_topics(10)

[(0,
  '0.197*"карта" + 0.135*"кредит" + 0.117*"деньга" + 0.111*"заявление" + 0.108*"счёт" + 0.103*"банкомат" + 0.102*"вклад" + 0.102*"отделение" + 0.098*"сбербанк" + 0.098*"сумма"'),
 (1,
  '0.366*"кредит" + -0.361*"вклад" + -0.226*"банкомат" + -0.180*"карта" + 0.169*"страховка" + 0.145*"задолженность" + 0.142*"платёж" + 0.140*"погашение" + -0.125*"деньга" + 0.117*"звонок"'),
 (2,
  '-0.518*"вклад" + 0.270*"банкомат" + 0.209*"карта" + 0.182*"сбербанк" + 0.134*"заявление" + -0.121*"очередь" + 0.116*"операция" + -0.114*"кредит" + -0.114*"договор" + 0.113*"средство"'),
 (3,
  '-0.265*"вклад" + -0.219*"сумма" + 0.195*"номер" + -0.185*"руб" + 0.180*"заявка" + -0.159*"погашение" + -0.158*"платёж" + -0.155*"договор" + 0.150*"звонок" + -0.145*"рубль"'),
 (4,
  '0.319*"вклад" + 0.275*"номер" + -0.266*"карта" + -0.210*"страховка" + 0.195*"звонок" + -0.187*"заявка" + 0.143*"ваш" + 0.142*"телефон" + -0.138*"кредит" + -0.129*"лимит"'),
 (5,
  '0.480*"банкомат" + -0.361*"карта" + 0.158*"купюра" + 0

In [57]:
# выделить явные темы: 1)кредиты и проблемы с погашением долга, 
#2)вклады и обращение в поддержку по ним 
#3)банкомант и проблемы со снятием наличных и др.

In [62]:
""" '0.249*"сбербанк" + 0.243*"заявка" + 0.207*"заявление" + 0.194*"страховка" + -0.192*"платёж" + 0.150*"страхование" + 0.148*"документ" + -0.146*"задолженность" + 0.145*"рассмотрение" + 0.136*"вклад"'),
"""
#тема с конкретным банком(Сбербанк): скорее всего тема связана с заявлением на отказ от страховки в Сбере

' \'0.249*"сбербанк" + 0.243*"заявка" + 0.207*"заявление" + 0.194*"страховка" + -0.192*"платёж" + 0.150*"страхование" + 0.148*"документ" + -0.146*"задолженность" + 0.145*"рассмотрение" + 0.136*"вклад"\'),\n'